In [53]:
import json
import string
import requests
from requests.exceptions import Timeout
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from scipy.stats import linregress
import csv 
import io


In [54]:
fatality = pd.read_csv('data/accident.csv')
injury_property_damage = pd.read_csv('data/injury_and_property_damage_only.csv')
vehicle_miles = pd.read_csv('data/vehicle_miles.csv')
injury_property_damage

c:\Users\datqu\anaconda3\envs\bootcamp\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (28,30,32,58,64,66,86,88,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,caseyear,injury_only,property_damage_only
0,2010,1542104,3847045
1,2011,1529968,3777994
2,2012,1634180,3949858
3,2013,1591016,4065673
4,2014,1647726,4386502
5,2015,1715394,4548203
6,2016,2116308,4670073
7,2017,1888525,4529513
8,2018,1893704,4807058
9,2019,1916344,4806253


In [55]:
fatality.head()

,caseyear,state,st_case,statename,ve_total,ve_forms,pvh_invl,peds,pernotmvit,permvit,...,hosp_mn,hosp_mnname,cf1,cf1name,cf2,cf2name,cf3,cf3name,fatals,drunk_dr
0,2010,1,10001,Alabama,1,1,NaN,0,NaN,NaN,...,99,Unknown EMS Hospital Arrival Time,0.0,None,0.0,None,0.0,None,1,1
1,2010,1,10002,Alabama,1,1,NaN,0,NaN,NaN,...,99,Unknown EMS Hospital Arrival Time,0.0,None,0.0,None,0.0,None,1,0
2,2010,1,10003,Alabama,3,3,NaN,0,NaN,NaN,...,99,Unknown EMS Hospital Arrival Time,0.0,None,0.0,None,0.0,None,1,0
3,2010,1,10004,Alabama,2,2,NaN,0,NaN,NaN,...,88,Not Applicable (Not Transported),0.0,None,0.0,None,0.0,None,1,0
4,2010,1,10005,Alabama,2,2,NaN,0,NaN,NaN,...,88,Not Applicable (Not Transported),0.0,None,0.0,None,0.0,None,1,0


In [56]:
vehicle_miles.head()

,caseyear,vehicle_miles_milions,mode
0,2010,5975.78,Air
1,2011,6004.58,Air
2,2012,5956.17,Air
3,2013,5964.98,Air
4,2014,5947.35,Air


In [57]:
fatality.groupby(['statename']).sum()['fatals'].reset_index().to_csv('data/summary_of_fatal_car.csv',index='False')

In [58]:
fatal_case_by_year = fatality.groupby(['caseyear']).count()['fatals'].reset_index()
fatal_case_by_year

,caseyear,fatals
0,2010,30626
1,2011,30210
2,2012,31358
3,2013,30516
4,2014,30343
5,2015,32835
6,2016,35017
7,2017,34841
8,2018,34214
9,2019,33762


In [70]:
summary_crash_case = fatal_case_by_year.merge(injury_property_damage,on='caseyear')
summary_crash_case['total_crash'] = summary_crash_case.fatals + summary_crash_case.injury_only + summary_crash_case.property_damage_only
summary_crash_case = summary_crash_case.merge(vehicle_miles.loc[vehicle_miles['mode'] == 'Car',['caseyear','vehicle_miles_milions']],on='caseyear')
summary_crash_case['fatality_per_crash'] = summary_crash_case.fatals / summary_crash_case.total_crash 
summary_crash_case['fatality_per_vmt'] = summary_crash_case.fatals / summary_crash_case.vehicle_miles_milions 
summary_crash_case

,caseyear,fatals,injury_only,property_damage_only,total_crash,vehicle_miles_milions,fatality_per_crash,fatality_per_vmt
0,2010,30626,1542104,3847045,5419775,2967265.97,0.005651,0.010321
1,2011,30210,1529968,3777994,5338172,2950401.81,0.005659,0.010239
2,2012,31358,1634180,3949858,5615396,2969432.94,0.005584,0.010560
3,2013,30516,1591016,4065673,5687205,2988280.19,0.005366,0.010212
4,2014,30343,1647726,4386502,6064571,3025655.73,0.005003,0.010029
5,2015,32835,1715394,4548203,6296432,3095372.70,0.005215,0.010608
6,2016,35017,2116308,4670073,6821398,3174407.96,0.005133,0.011031
7,2017,34841,1888525,4529513,6452879,3212347.31,0.005399,0.010846
8,2018,34214,1893704,4807058,6734976,3240326.54,0.005080,0.010559
9,2019,33762,1916344,4806253,6756359,3261771.66,0.004997,0.010351


,caseyear,vehicle_miles_milions
11,2010,2967265.97
12,2011,2950401.81
13,2012,2969432.94
14,2013,2988280.19
15,2014,3025655.73
16,2015,3095372.70
17,2016,3174407.96
18,2017,3212347.31
19,2018,3240326.54
20,2019,3261771.66
